In [2]:
# grover_practice.py
"""
GROVER'S ALGORITHM PRACTICE
Build from scratch, understand each piece
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import numpy as np

print("🔍 GROVER'S ALGORITHM PRACTICE SESSION")
print("=" * 60)

# ==================== STEP 1: 2-QUBIT GROVER ====================

def build_oracle_2qubit(target):
    """
    Build oracle for 2-qubit system
    target: '00', '01', '10', or '11'
    """
    qc = QuantumCircuit(2)
    
    # Apply X gates to qubits that are 0 in target
    # This makes target temporarily |11⟩ so CZ can mark it
    if target[1] == '0':  # Qubit 0 (rightmost in string)
        qc.x(0)
    if target[0] == '0':  # Qubit 1 (leftmost in string)
        qc.x(1)
    
    # Apply CZ to mark |11⟩
    qc.cz(0, 1)
    
    # Apply X gates again to return to original
    if target[1] == '0':
        qc.x(0)
    if target[0] == '0':
        qc.x(1)
    
    return qc

def build_diffuser_2qubit():
    """Standard diffuser for 2 qubits"""
    qc = QuantumCircuit(2)
    
    # H on both
    qc.h(0)
    qc.h(1)
    
    # X on both
    qc.x(0)
    qc.x(1)
    
    # CZ (same as oracle pattern)
    qc.cz(0, 1)
    
    # X on both again
    qc.x(0)
    qc.x(1)
    
    # H on both again
    qc.h(0)
    qc.h(1)
    
    return qc

def grover_2qubit(target, iterations=1):
    """Complete Grover for 2 qubits"""
    print(f"\n🎯 Searching for |{target}⟩ in 4 items")
    print(f"   Iterations: {iterations}")
    print("-" * 40)
    
    # Build circuit
    qc = QuantumCircuit(2, 2)
    
    # Step 1: Initialize superposition
    qc.h(0)
    qc.h(1)
    
    # Step 2: Apply iterations
    for i in range(iterations):
        # Add oracle
        oracle = build_oracle_2qubit(target)
        qc.compose(oracle, inplace=True)
        
        # Add diffuser
        diffuser = build_diffuser_2qubit()
        qc.compose(diffuser, inplace=True)
        
        print(f"   Applied iteration {i+1}")
    
    # Step 3: Measure
    qc.measure([0, 1], [0, 1])
    
    return qc

def test_2qubit_grover():
    """Test all 2-qubit cases"""
    targets = ['00', '01', '10', '11']
    
    for target in targets:
        qc = grover_2qubit(target, iterations=1)
        
        # Run simulation
        simulator = AerSimulator()
        compiled = transpile(qc, simulator)
        job = simulator.run(compiled, shots=100)
        result = job.result()
        counts = result.get_counts()
        
        # Find success rate
        success = counts.get(target, 0)
        success_rate = (success / 100) * 100
        
        print(f"   Success rate: {success_rate:.1f}%")
        print(f"   Random chance: 25.0%")
        
        if success_rate > 70:
            print("   ✅ Good amplification!")
        else:
            print("   ⚠️  Needs more iterations")
        print()

# ==================== STEP 2: UNDERSTANDING THE DIFFUSER ====================

def analyze_diffuser_effect():
    """See what the diffuser actually does"""
    print("\n" + "="*60)
    print("🔬 ANALYZING THE DIFFUSER")
    print("="*60)
    
    # Create a simple test state: marked |11⟩
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.h(1)
    
    # Mark |11⟩ with oracle
    qc.cz(0, 1)  # This marks |11⟩ with -1 phase
    
    print("\nState BEFORE diffuser (after oracle marks |11⟩):")
    state_before = Statevector.from_instruction(qc)
    for i in range(4):
        print(f"  |{i:02b}⟩: {state_before[i]:.3f}")
    
    # Apply diffuser
    diffuser = build_diffuser_2qubit()
    qc.compose(diffuser, inplace=True)
    
    print("\nState AFTER diffuser:")
    state_after = Statevector.from_instruction(qc)
    for i in range(4):
        print(f"  |{i:02b}⟩: {state_after[i]:.3f}")
    
    print("\n📈 Probability changes:")
    for i in range(4):
        before_prob = abs(state_before[i])**2
        after_prob = abs(state_after[i])**2
        change = after_prob - before_prob
        print(f"  |{i:02b}⟩: {before_prob:.3f} → {after_prob:.3f} (Δ={change:+.3f})")

# ==================== STEP 3: 3-QUBIT GROVER ====================

def build_oracle_3qubit(target):
    """Oracle for 3-qubit system"""
    qc = QuantumCircuit(3)
    
    # Apply X to qubits that are 0 in target
    # target format: '010' means q2=0, q1=1, q0=0
    for i in range(3):
        if target[2-i] == '0':  # Reverse index for Qiskit convention
            qc.x(i)
    
    # Apply CCZ to mark |111⟩
    qc.h(2)
    qc.ccx(0, 1, 2)  # Toffoli = controlled-controlled-X
    qc.h(2)
    
    # Apply X gates again
    for i in range(3):
        if target[2-i] == '0':
            qc.x(i)
    
    return qc

def build_diffuser_3qubit():
    """Diffuser for 3 qubits"""
    qc = QuantumCircuit(3)
    
    # H on all
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    # X on all
    qc.x(0)
    qc.x(1)
    qc.x(2)
    
    # CCZ (triple-controlled Z)
    qc.h(2)
    qc.ccx(0, 1, 2)
    qc.h(2)
    
    # X on all again
    qc.x(0)
    qc.x(1)
    qc.x(2)
    
    # H on all again
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    return qc

def grover_3qubit_example():
    """Example for 3-qubit search (8 items)"""
    print("\n" + "="*60)
    print("🧩 3-QUBIT GROVER (Searching 8 items)")
    print("="*60)
    
    target = '101'  # Searching for |101⟩
    print(f"Target: |{target}⟩")
    
    # Build circuit
    qc = QuantumCircuit(3, 3)
    
    # Initialize
    qc.h(0); qc.h(1); qc.h(2)
    
    # Oracle
    oracle = build_oracle_3qubit(target)
    qc.compose(oracle, inplace=True)
    
    # Diffuser
    diffuser = build_diffuser_3qubit()
    qc.compose(diffuser, inplace=True)
    
    # One more iteration (total 2 for 8 items)
    qc.compose(oracle, inplace=True)
    qc.compose(diffuser, inplace=True)
    
    # Measure
    qc.measure([0, 1, 2], [0, 1, 2])
    
    # Run
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    job = simulator.run(compiled, shots=100)
    result = job.result()
    counts = result.get_counts()
    
    print("\nResults:")
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    for state, count in sorted_counts[:5]:  # Top 5 results
        percentage = (count / 100) * 100
        if state == target:
            print(f"  ✅ |{state}⟩: {count} times ({percentage:.1f}%) ← TARGET!")
        else:
            print(f"     |{state}⟩: {count} times ({percentage:.1f}%)")
    
    success_rate = (counts.get(target, 0) / 100) * 100
    print(f"\n🎯 Success rate: {success_rate:.1f}%")
    print(f"🎲 Random chance: 12.5% (1/8)")

# ==================== STEP 4: YOUR TURN TO PRACTICE ====================

def practice_exercises():
    """Exercises for you to complete"""
    print("\n" + "="*60)
    print("💪 YOUR TURN: PRACTICE EXERCISES")
    print("="*60)
    
    exercises = [
        {
            "level": "Beginner",
            "task": "Complete the oracle for |01⟩ in 2-qubit system",
            "hint": "Only qubit 0 needs X gates (target '01' means q0=0, q1=1)",
            "function": None  # You'll implement this
        },
        {
            "level": "Intermediate",
            "task": "Calculate optimal iterations for 16 items",
            "hint": "Formula: π/4 × √N. N=16 → √16=4 → π/4×4≈3.14 → round to 3",
            "answer": "3 iterations"
        },
        {
            "level": "Advanced",
            "task": "Modify Grover to search for TWO items simultaneously",
            "hint": "Oracle should mark both |01⟩ AND |10⟩ with -1 phase",
            "challenge": "Build oracle marking multiple states"
        }
    ]
    
    for i, ex in enumerate(exercises, 1):
        print(f"\nExercise {i} ({ex['level']}):")
        print(f"  {ex['task']}")
        print(f"  💡 Hint: {ex['hint']}")
        
        if i == 1:
            print("\n  Your code here:")
            print("  def oracle_for_01():")
            print("      qc = QuantumCircuit(2)")
            print("      # Apply X to qubit _____")
            print("      qc.cz(0, 1)")
            print("      # Apply X to qubit _____")
            print("      return qc")
    
    return exercises

def visual_grover_explanation():
    """Visual explanation of Grover's amplification"""
    print("\n" + "="*60)
    print("📊 VISUALIZING GROVER'S AMPLIFICATION")
    print("="*60)
    
    print("\nImagine 4 states with equal probability:")
    print("  |00⟩: ████████████████ 25%")
    print("  |01⟩: ████████████████ 25%")
    print("  |10⟩: ████████████████ 25%")
    print("  |11⟩: ████████████████ 25%")
    
    print("\nAfter oracle marks |11⟩ (adds negative phase):")
    print("  |00⟩: ████████████████ 25%")
    print("  |01⟩: ████████████████ 25%")
    print("  |10⟩: ████████████████ 25%")
    print("  |11⟩: ████████░░░░░░░░ 25% but with - sign")
    
    print("\nAfter diffuser (inversion about average):")
    print("  |00⟩: ████████░░░░░░░░ ~6%")
    print("  |01⟩: ████████░░░░░░░░ ~6%")
    print("  |10⟩: ████████░░░░░░░░ ~6%")
    print("  |11⟩: ████████████████████████ ~94%")
    
    print("\n🎯 The marked state gets amplified!")

# ==================== MAIN ====================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════╗
    ║        GROVER'S ALGORITHM PRACTICE       ║
    ║    Search Faster with Quantum Magic      ║
    ╚══════════════════════════════════════════╝
    """)
    
    # Run 2-qubit tests
    print("\n🔍 TESTING 2-QUBIT GROVER (All targets)")
    test_2qubit_grover()
    
    # Analyze diffuser
    analyze_diffuser_effect()
    
    # 3-qubit example
    grover_3qubit_example()
    
    # Visual explanation
    visual_grover_explanation()
    
    # Practice exercises
    practice_exercises()
    
    print("\n" + "="*60)
    print("🎓 KEY TAKEAWAYS:")
    print("="*60)
    print("1. Oracle = marks solution with -1 phase")
    print("2. Diffuser = amplifies marked states")
    print("3. Iterations ≈ π/4 × √N")
    print("4. Speedup: Classical O(N) → Quantum O(√N)")
    print("\n💡 Remember: Grover doesn't tell you WHAT the solution is,")
    print("   it helps you FIND a solution you can recognize!")

🔍 GROVER'S ALGORITHM PRACTICE SESSION

    ╔══════════════════════════════════════════╗
    ║        GROVER'S ALGORITHM PRACTICE       ║
    ║    Search Faster with Quantum Magic      ║
    ╚══════════════════════════════════════════╝
    

🔍 TESTING 2-QUBIT GROVER (All targets)

🎯 Searching for |00⟩ in 4 items
   Iterations: 1
----------------------------------------
   Applied iteration 1
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Good amplification!


🎯 Searching for |01⟩ in 4 items
   Iterations: 1
----------------------------------------
   Applied iteration 1
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Good amplification!


🎯 Searching for |10⟩ in 4 items
   Iterations: 1
----------------------------------------
   Applied iteration 1
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Good amplification!


🎯 Searching for |11⟩ in 4 items
   Iterations: 1
----------------------------------------
   Applied iteration 1
   Success rate: 100.0%
   Random chance: 